## Selecting the epitopes for further calculations
In this file the parsed data of the VDJ and McPAS databases are merged together with the negative data, negative data entries were only used if there CDR3 sequences were not present in the VDJ or McPAS database.

In further calculations only epitopes present more than 30 times were used, this because they are used both for training and testing of the random forrest classifier in the next steps, epitopes which were present less than 30 times were not suited for this.

A short summary of this data is made, for each epitope the number of entries are given together with the number of unique CDR3 sequences, the CDR3 diversity and the number of unique V and J genes. 

In the last part there are always 2 or 3 epitopes selected together with a part of the negative data, this part of the data is than futher used in the 'Classification' file in a random forrest classifier. 

In [7]:
import pandas as pd
import collections
from skbio.diversity.alpha import shannon 
import IPython

In [12]:
# import of parsed files, for some reason the columns V_family and J_gene were changed to float variables while importing so I changed it back, I did not find why this happened
McPAS_data = pd.read_csv("parsed_McPAS.csv")
McPAS_data['J_gene'] = McPAS_data['J_gene'].astype(int).astype(str)
McPAS_data['V_family'] = McPAS_data['V_family'].astype(int).astype(str)
McPAS_data['J_gene'] = McPAS_data['J_gene'].str.zfill(2)
McPAS_data['V_family'] = McPAS_data['V_family'].str.zfill(2)

vdjdb_data = pd.read_csv("parsed_vdjdb.csv")
vdjdb_data['J_gene'] = vdjdb_data['J_gene'].astype(int).astype(str)
vdjdb_data['V_family'] = vdjdb_data['V_family'].astype(int).astype(str)
vdjdb_data['J_gene'] = vdjdb_data['J_gene'].str.zfill(2)
vdjdb_data['V_family'] = vdjdb_data['V_family'].str.zfill(2)

control_data = pd.read_csv("parsed_neg.csv")
control_data['J_gene'] = control_data['J_gene'].astype(int).astype(str)
control_data['V_family'] = control_data['V_family'].astype(int).astype(str)
control_data['J_gene'] = control_data['J_gene'].str.zfill(2)
control_data['V_family'] = control_data['V_family'].str.zfill(2)

# Remove CDR3 sequences from negative data if they also occur in the positive data
a = McPAS_data['CDR3']
b = vdjdb_data['CDR3']
control = control_data[~control_data['CDR3'].isin(a)]
control = control_data[~control_data['CDR3'].isin(b)]


In [13]:
data = pd.concat([McPAS_data, vdjdb_data,control], ignore_index=True)
data = data.drop_duplicates()
# delete epitopes if they are present less then 30 times
data = data.groupby('Epitope').filter(lambda x: len(x)> 30)
classes = data['Epitope'].unique()
data.head()

,CDR3,Epitope,Gene,J_gene,V_family,V_gene
5,CAVSDITYKYIF,AVFDRKSDAK,TRA,40,16,16-01
6,CAEYSSASKIIF,GLCTLVAML,TRA,03,15,15-01
7,CAEDADSTLTF,GLCTLVAML,TRA,11,15,15-01
8,CAESTSGGKLIF,GLCTLVAML,TRA,23,15,15-01
9,CAESTGKLIF,GLCTLVAML,TRA,23,15,15-01


In [14]:
data_1.shape

(84638, 6)

In [15]:
# calculate CDR3 sequence diversity of input data
stats = collections.defaultdict(list)
for peptide in classes:
    pep_data = data[data['Epitope'] == peptide]
    
    stats['total TCRAs'].append(pep_data.shape[0])
    stats['unique CDR3'].append(len(set(pep_data['CDR3'])))
    
    cdr3_count = list(collections.Counter(pep_data['CDR3']).values())
    stats['CDR3 diversity'].append(shannon(cdr3_count))

    stats['unique V gene'].append(len(pep_data['V_gene'].unique()))
    stats['unique J gene'].append(len(pep_data['J_gene'].unique()))

IPython.display.display(pd.DataFrame(stats, index=classes, columns=stats.keys()).T)

,AVFDRKSDAK,GLCTLVAML,NLVPMVATV,LLWNGPMAV,GILGFVFTL,ELAGIGILTV,NEGVKAAW,KLGGALQAK,IVTDFSVIK,RAKFKQLL,RLRAEAQVK,CINGVCWTV,GLIYNRMGAVTTEV,PKYVKQNTLKLAT,QARQMVQAMRTIGTHP,LLLGIGILV,DATYQRTRALVR,Control
total TCRAs,562.000000,69.000000,876.000000,143.000000,1037.000000,135.000000,46.000000,4395.000000,193.000000,255.000000,157.000000,34.000000,41.000000,54.000000,53.00000,125.000000,32.0,76431.00000
unique CDR3,558.000000,59.000000,839.000000,137.000000,921.000000,131.000000,45.000000,4278.000000,192.000000,252.000000,157.000000,34.000000,41.000000,54.000000,53.00000,123.000000,32.0,76431.00000
CDR3 diversity,9.120191,5.807729,9.686306,7.075955,9.786629,7.017556,5.480084,12.042955,7.582094,7.970824,7.294621,5.087463,5.357552,5.754888,5.72792,6.933784,5.0,16.22187
unique V gene,42.000000,17.000000,45.000000,23.000000,43.000000,18.000000,19.000000,43.000000,40.000000,40.000000,38.000000,11.000000,20.000000,27.000000,14.00000,28.000000,12.0,46.00000
unique J gene,35.000000,26.000000,44.000000,33.000000,49.000000,29.000000,18.000000,41.000000,29.000000,30.000000,27.000000,16.000000,18.000000,23.000000,17.00000,27.000000,16.0,54.00000


In [16]:
data_1 = data.loc[data['Epitope'] == 'AVFDRKSDAK']
data_2 = data.loc[data['Epitope'] == 'CINGVCWTV']

neg_data = control.sample(6000)
data = pd.concat([neg_data, data_1, data_2], ignore_index=True)

data.head()

,CDR3,Epitope,Gene,J_gene,V_family,V_gene
0,CAGPRGGKTPLVF,Control,TRA,29,25,25-01
1,CVTEYSGTTDSWGKLQF,Control,TRA,24,30,30-01
2,CAMTLPISSGSTRQLTF,Control,TRA,22,12,12-03
3,CAVAAGLTSYGKLTF,Control,TRA,52,08,08-01
4,CADAAGGGADGLTF,Control,TRA,45,35,35-01


In [17]:
data.to_csv('data.csv', index=False, sep=",")

In [18]:
data.shape

(6596, 6)